# Discretizing GeoLife Trajectory 1.3 sample data into a spatiotemporal grid

*Part of the COVID19risk project*  
*http://covid19risk.com/*  
*2020-03-04*  

*Copyright (C) 2020 Mikhail Voloshin, Mighty Data Inc.*  
*All rights reserved.*  

The objective of this Notebook is to convert the sample GeoLife trajectory data from Microsoft (https://www.microsoft.com/en-us/download/details.aspx?id=52367) into a form that can be marshalled to a browser for purposes of being rendered client-side as a heatmap that can be navigated across space and time.

The GeoLife data is hundreds of megabytes ZIPped, and over 1.5 GB expanded into CSV-based "trajectory" files. This is far too big and too detailed to be usable by a client.

As such, the purpose of this Notebook is to create a "summary" of the data that is optimized for rendering as a geospatial map, such as an overlap atop Google Maps.

This is achieved via the following approach:
 1. The user specifies a spatial resolution, in kilometers (default 5). Note that, because the Earth is round, this will only be approximately accurate. It will be optimized for best performance near the central US. Cells will grow finer (smaller, so fewer data points fall into each cell) closer to the poles, and coarser (larger, so more data points fall into individual cells) closer to the equator. Later adaptations of this software can compute cell boundaries more accurately.
 1. The user specifies a temporal resolution, in days (default 7)
 1. The user specifies a data directory into which the Microsoft Research GeoLife ZIP file got unzipped. If the directory name is the same as the filename, then this directory will be called "Geolife Trajectories 1.3"
 1. This script will discretize the trajectory data into this grid. This script will iterate over every data point in every trajectory in the data directory, determine which spatiotemporal grid cell the point falls into, and tally the total number of unique trajectories (individuals) who appear in each cell.
 1. The resultant tallies will be output as a time series of heatmaps. Each heatmap is inferred to be sparse, so it will be represented as a map; keys representing the latitude and longitude of each cell map to the number of trajectories tallied in that cell for that time interval.
 

## Define parameters

In [1]:
# Define parameters

# The number of kilometers covered by each grid cell, in both the latitude and longitude direction.
# Discretization is zeroed on the equator and prime meridian for latitude and longitude respectively.
# For now, the number of kilometers per degree is fixed at a conversion rate optimized for 40oN -100oW,
# which lies in northern Kansas near the geospatial center of the US. Future versions of this
# script will employ an algorithm that takes the Earth's curvature into account. For now, this will lead
# to a slight illusory increase in data points per cell (i.e. deceptively "hotter" cells) near
# the equator, and an illusory decrease in data points per cell (i.e. deceptively "colder" cells)
# near the poles. The distortion near the poles will be significant, but because those are far from
# human population centers this should be acceptable.
GRID_SPATIAL_KM = 5

# The number of days for each time step into which to discretize the data.
# Discretization is zeroed on the Unix epoch.
GRID_TEMPORAL_DAYS = 7

# Path to the unzipped Geolife data folder.
DATA_DIR = "../../../../data/proof-of-concept/Geolife Trajectories 1.3"

## Search for trajectory files

In [2]:
import os

if not os.path.isdir(DATA_DIR):
    raise ValueError(DATA_DIR + " doesn't appear to be a directory that exists on this filesystem.")
if not os.path.isdir(DATA_DIR+'/Data'):
    raise ValueError(DATA_DIR + " doesn't appear to be the unzipped Geolife dataset. It doesn't contain a /Data subdirectory.")

# A trajectory folder path is any subfolder of the Data directory that has a Trajectory subfolder.
trajectory_folder_paths = [f.path for f in os.scandir(DATA_DIR+'/Data') if f.is_dir() and os.path.isdir(f.path+'/Trajectory')]

print(f'Success. {len(trajectory_folder_paths)} trajectories found.')

Success. 182 trajectories found.


## Initialize the data structures we'll output

In [3]:
cell_spatiotemporal_tallies = {}

## Precompute some values that will be useful to us during the computation

Spatial grid degrees were computed with the help of https://andrew.hedges.name/experiments/haversine/. Future versions of this script should use trigonometry to discretize by traversing a longitudinal line in intervals of *GRID_SPATIAL_KM* km steps from the equator, and then using the Earth's cross-section at that latitude to walk by *GRID_SPATIAL_KM* km steps from the prime meridian. It's not hard, but a little too tedious to go through the trouble of writing said function for this proof-of-concept stage.

In [4]:
CELLSIZE_SECONDS = GRID_TEMPORAL_DAYS * 24 * 60 * 60

CELLSIZE_DEGREES_LAT = GRID_SPATIAL_KM * 0.00899
CELLSIZE_DEGREES_LONG = GRID_SPATIAL_KM * 0.01174
CELLSIZE_DEGREES_PRECISION = 5


## Define some helper functions

In [5]:
import time
import datetime

MICROSOFT_EPOCH_START = time.mktime(datetime.datetime.strptime('1899-12-30', '%Y-%m-%d').timetuple())

def convert_microsoft_epoch_to_unixtime(dayscount):
    # Microsoft Research encoded this dataset to include a field
    # that contains the number of days that have elapsed since
    # Dec 30 (not 31), 1899. No word on whether this is midnight
    # at the *beginning* or *end* of said date, but we can validate
    # against the other date and time columns to make sure we
    # got it right (which we did).

    secondscount = dayscount * 24 * 60 * 60
    retval = MICROSOFT_EPOCH_START + secondscount
    return int(retval)
    


In [6]:
latlong_fstring_part = f'{{:.{CELLSIZE_DEGREES_PRECISION}f}}'
latlong_fstring = f'{latlong_fstring_part},{latlong_fstring_part}'

def determine_grid_cell(df):
    df['unixtime'] = df['daysSinceMicrosoftEpoch'].apply(convert_microsoft_epoch_to_unixtime)    
    
    # We're using int as a de facto math.floor function
    df['cell_latitude'] = (df['latitude'] / CELLSIZE_DEGREES_LAT).apply(int) * CELLSIZE_DEGREES_LAT
    df['cell_longitude'] = (df['longitude'] / CELLSIZE_DEGREES_LONG).apply(int) * CELLSIZE_DEGREES_LONG
    df['cell_unixtime'] = (df['unixtime'] / CELLSIZE_SECONDS).apply(int) * CELLSIZE_SECONDS

    df['cell_key_spatial'] = df.apply(lambda x: latlong_fstring.format(x['cell_latitude'], x['cell_longitude']), axis=1)
    df['cell_key_spatiotemporal'] = df.apply(lambda x: '{},{}'.format(x['cell_key_spatial'], x['cell_unixtime']), axis=1)
    
    return df
    

## Populate our data structures

In [7]:
%%time

import pandas as pd

# The column meanings come from the User Guide PDF that comes with the Geolife dataset.
GEOLIFE_COLUMNS = [
    'latitude',
    'longitude',
    'reserved0',
    'altitude',
    'daysSinceMicrosoftEpoch',
    'date',
    'time'    
]


for traj_path in trajectory_folder_paths:
    traj_id = traj_path.split('/')[-1]
    print(f'Processing Trajectory ID: {traj_id}')

    cells_hit_by_this_traj = set()
    
    traj_filenames = [f.path for f in os.scandir(traj_path+'/Trajectory') if f.is_file() and f.path.endswith('.plt')]
    print(f'{len(traj_filenames)} trajectory plots found.')
    
    for traj_filename in traj_filenames:
        df = pd.read_csv(traj_filename, skiprows=6, names=GEOLIFE_COLUMNS)
        determine_grid_cell(df)
        cells_hit = set(df['cell_key_spatiotemporal'].unique())
        cells_hit_by_this_traj.update(cells_hit)
        print(".", end="")
    
    for cellkey in cells_hit_by_this_traj:
        if cellkey not in cell_spatiotemporal_tallies:
            cell_spatiotemporal_tallies[cellkey] = 1
        else:
            cell_spatiotemporal_tallies[cellkey] += 1

            
    print(f'\nTrajectory ID {traj_id} done.\n')
    break

Processing Trajectory ID: 000
171 trajectory plots found.
...........................................................................................................................................................................
Trajectory ID 000 done.

CPU times: user 8.17 s, sys: 531 ms, total: 8.7 s
Wall time: 8.64 s


In [8]:
len(cell_spatiotemporal_tallies)

190